In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)
z = x + sym.I*y
f = lambda z: z**3 - 1

F = [sym.re(f),sym.im(f)]

def GetJacobian(f,r,h=1e-6):
    n = r.shape[0]
    J = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            rf = r.copy()
            rb = r.copy()
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            J[i,j] = ( f[i](*rf) - f[i](*rb)  )/(2*h)
    

    F_num = sym.lambdify([x, y], F, 'numpy')
    J_num = sym.lambdify([x, y], J, 'numpy')
    
    return J, J_num, F_num

def NewtonRaphson(Fn, z0, Jn, itmax=1000,epsilon=1e-7):
    zn = z0
    error = float('inf')
    it = 0

    while error > epsilon and it < itmax:
        Fz = np.array(Fn(*zn))
        Jz = np.array(Jn(*zn))
        invJz = np.linalg.pinv(Jz)  # Pseudo-inversa para lidiar con puntos cercanos a z=0

        zn_new = zn - np.dot(invJz, Fz)
        error = np.linalg.norm(zn_new - zn)
        zn = zn_new
        it += 1

    

    z_0 = [0.5, 0,5]
    respuesta = NewtonRaphson(Fn, z_0, Jn)
    
    return respuesta

N = 300
x = np.linspace(-1, 1, N)
y = np.linspace(-1, 1, N)
fractal = np.zeros((N, N), np.int64)
z0 = -1/2 + np.sqrt(3)/2
z1 = -1/2 - np.sqrt(3)/2
z2 = 1 
for i, real in enumerate(x):
    for j, imag in enumerate(y):
        z = (real, imag)
        raiz = NewtonRaphson(Fn, z, Jn)
        if  round(raiz[0] + raiz[1], 4) == round(z0, 4):
            fractal[i, j] = 20
        elif round(raiz[0] + raiz[1], 4) == round(z1, 4):
            fractal[i, j] = 100
        elif round(raiz[0] + raiz[1], 4) == round(z2, 4):
            fractal[i, j] = 255
plt.imshow(fractal, cmap='coolwarm' ,extent=[-1,1,-1,1])